In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Slake knowledge base

In [ ]:
!unzip /content/drive/MyDrive/data/Slake.zip -d /content

In [ ]:
!cat /content/Slake1.0/KG/en_disease.csv

In [ ]:
kgs1 = []
for line in open('/content/Slake1.0/KG/en_disease.csv'):
  kgs1.append(line.lower().replace('#',' ').strip('\n'))

In [ ]:
kgs2 = []
for line in open('/content/Slake1.0/KG/en_organ.csv'):
  kgs2.append(line.lower().replace('#',' ').strip('\n'))

In [ ]:
kgs = kgs1[1:] + kgs2[1:]

In [ ]:
kgs

['alveolar proteinosis location lung',
 'alveolar proteinosis description a rare disease characterized by positive pas staining in the alveolar and bronchiolar lumens and phospholipid-rich proteins from the lungs, occurs in young and middle-aged adults, with about three times as many males as females.',
 'alveolar proteinosis prevention avoid bacterial infection and improve immunity',
 'alveolar proteinosis cause decreased immune function, genetic factors, alcohol abuse, microbial infection',
 'alveolar proteinosis symptom cyanosis, chest pain, dyspnea, fatigue',
 'alveolar proteinosis treatment bronchoalveolar lavage',
 'alveolar proteinosis infectivity non-infectious',
 'pertussis location respiratory system',
 'pertussis description acute respiratory infectious diseases caused by pertussis bacilli',
 'pertussis prevention vaccination',
 'pertussis cause bordetella pertussis',
 'pertussis symptom spastic cough',
 'pertussis treatment medication',
 'pertussis infectivity respiratory t

In [ ]:
import pandas as pd

df1 = pd.read_csv('/content/Slake1.0/KG/en_disease.csv', sep='#')
df1

,disease,attribute,value
0,Alveolar Proteinosis,location,Lung
1,Alveolar Proteinosis,description,a rare disease characterized by positive pas s...
2,Alveolar Proteinosis,prevention,avoid bacterial infection and improve immunity
3,Alveolar Proteinosis,cause,"decreased immune function, genetic factors, al..."
4,Alveolar Proteinosis,symptom,"cyanosis, chest pain, dyspnea, fatigue"
...,...,...,...
2211,Pneumothorax,prevention,"quit smoking, avoid strenuous exercise"
2212,Pneumothorax,cause,"chest injury, lung disease, bullae"
2213,Pneumothorax,symptom,"chest pain, dyspnea"
2214,Pneumothorax,treatment,"medical therapy, supportive therapy"


In [ ]:
df1.to_string

<bound method DataFrame.to_string of                    disease  ...                                              value
0     Alveolar Proteinosis  ...                                               Lung
1     Alveolar Proteinosis  ...  a rare disease characterized by positive pas s...
2     Alveolar Proteinosis  ...     avoid bacterial infection and improve immunity
3     Alveolar Proteinosis  ...  decreased immune function, genetic factors, al...
4     Alveolar Proteinosis  ...             cyanosis, chest pain, dyspnea, fatigue
...                    ...  ...                                                ...
2211          Pneumothorax  ...             quit smoking, avoid strenuous exercise
2212          Pneumothorax  ...                 chest injury, lung disease, bullae
2213          Pneumothorax  ...                                chest pain, dyspnea
2214          Pneumothorax  ...                medical therapy, supportive therapy
2215          Pneumothorax  ...                   

In [ ]:
import pandas as pd

df2 = pd.read_csv('/content/Slake1.0/KG/en_organ.csv', sep='#')
df2

,organ,attribute,value
0,Motor System,component,"bone, bone junctions, skeletal muscles"
1,Motor System,function,movement and support
2,Skeleton,definition,a structure used to support something
3,Joint,definition,connecting points between bone fragments of hu...
4,Joint,component,"joint capsule, articular surface, articular ca..."
...,...,...,...
276,Vestibule Of Ear,function,relevant to maintaining body balance
277,Semicircular Canal,function,maintain posture and balance
278,Semicircular Canal,definition,inner ear sensory apparatus
279,Parotid,function,"secrete saliva, secrete enzymes"


# Read questions

In [ ]:
!mkdir data
###### change to your own dataset file
!cp /content/drive/MyDrive/data/jsons/QA.json /content/data

In [ ]:
import numpy as np
import json
import re
import os


with open('/content/data/QA.json', 'r', encoding='utf-8') as f:
    data = json.load(f)


path = '/content/data/knowledge_embeddings'
if not os.path.exists(path):
    os.mkdir(path)


In [ ]:
len(data)

32761

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def ectract_query(ques):
  qs = nlp(ques)
  queries = []
  for tk in qs:
    # print(tk.text,'|',tk.pos_)
    if (tk.pos_ == 'NOUN') and tk.lemma_ not in queries:
      queries.append(tk.lemma_)
  return queries


In [ ]:
def retrieve_kn(q):
  kg = []
  for k in kgs:
    if (q in k):
      kg.append(k)
  return kg

In [ ]:
f = open('/content/knowledge.txt','a')
kg = []

for d in data:
  ques = d['Questions']
  id = d['Question_Id']
  queries = ectract_query(ques)
  # print(ques,queries)
  tmp = []
  for q in queries:
    tmp += retrieve_kn(q)
  # kg.append(tmp)
  kn_concat = ''
  for k in tmp[:10]:
    marked_text1 = k + "#"
    kn_concat += marked_text1
  kn_concat = str(id) + '#' + kn_concat[:-1] + '\n'
  f.write(kn_concat)
f.close()

In [ ]:
len(kg)

0

In [ ]:
cnt0 = 0
cnt10 = 0
cnt = 0
for k in kg:
  if len(k) == 0:
    cnt0 += 1
  elif len(k) > 10:
    cnt10 += 1
  else:
    cnt += 1

In [ ]:
print(cnt0,cnt10,cnt)

0 0 0


In [ ]:
a = [0,1,2,3,4,5,6]
for i in a[:5]:
  print(i)

0
1
2
3
4


In [ ]:
kn_data = []
for line in open('/content/knowledge.txt'):
  kn_data.append(line)

In [ ]:
kn_l = []
ma = 0
for d in kn_data:
  # if len(d.split()) > ma:
  #   ma = len(d.split())
  # print(len(d.split()))
  kn_l.append(len(d.split()))

In [ ]:
len(kn_l)

32761

In [ ]:
dl = {}
for l in kn_l:
  try:
    dl[l] += 1
  except:
    dl[l] = 0
print(sorted(dl.items(), key=lambda x:x[1])[::-1])

[(1, 13889), (46, 3566), (39, 1129), (88, 707), (124, 386), (172, 360), (125, 310), (222, 277), (87, 268), (265, 247), (233, 238), (120, 222), (91, 220), (105, 208), (140, 207), (174, 196), (84, 192), (93, 183), (90, 182), (38, 174), (122, 167), (99, 161), (86, 160), (132, 159), (129, 158), (144, 145), (162, 145), (176, 144), (81, 143), (109, 138), (73, 138), (100, 137), (118, 134), (119, 130), (134, 129), (75, 123), (194, 122), (165, 119), (96, 119), (202, 116), (242, 115), (178, 111), (130, 109), (79, 108), (117, 105), (83, 100), (108, 98), (21, 96), (185, 91), (25, 88), (94, 87), (143, 84), (136, 84), (98, 80), (128, 78), (101, 76), (82, 75), (106, 74), (116, 74), (148, 73), (138, 73), (141, 73), (68, 71), (64, 71), (139, 70), (249, 68), (112, 67), (14, 66), (77, 66), (92, 66), (27, 64), (164, 62), (137, 60), (177, 59), (149, 59), (55, 58), (74, 58), (85, 58), (23, 57), (303, 56), (229, 56), (103, 56), (89, 55), (126, 55), (37, 53), (156, 53), (175, 53), (28, 53), (244, 52), (133, 5

In [ ]:
avg = 0
for l,n in dl.items():
  avg += l*n/len(kn_l)
print(avg)

59.76154574036201


In [ ]:
ma

303

In [ ]:
!cp /content/knowledge_slakekg2.txt /content/drive/MyDrive/data